##### Copyright 2021 The TensorFlow Federated Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 学習に推奨される集計の調整

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/federated/tutorials/tuning_recommended_aggregators"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a></td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/federated/tutorials/tuning_recommended_aggregators.ipynb">     <img src="https://www.tensorflow.org/images/colab_logo_32px.png">     Google Colab で実行</a>
</td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/federated/tutorials/tuning_recommended_aggregators.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"> GitHub でソースを表示</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/federated/tutorials/tuning_recommended_aggregators.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
</table>

`tff.learning` モジュールには、モデルの更新を集約するためのいくつかの方法が含まれています。推奨される構成はデフォルトです。

- `tff.learning.robust_aggregator`
- `tff.learning.dp_aggregator`
- `tff.learning.compression_aggregator`
- `tff.learning.secure_aggregator`

このチュートリアルでは、根本的な動機とその実装方法を説明し、構成をカスタマイズする方法について提案します。 

---

In [ ]:
#@test {"skip": true}
!pip install --quiet --upgrade tensorflow-federated

In [1]:
import math
import tensorflow_federated as tff
tff.federated_computation(lambda: 'Hello, World!')()

b'Hello, World!'

集計メソッドは、`model_aggregator` キーワード引数として `tff.learning.algorithms.build_weighted_fed_avg`（および `build_unweighted_fed_avg`）に渡すことができるオブジェクトで表されます。ここで説明するアグリゲータを直接使用して、連合学習の[前の](federated_learning_for_image_classification.ipynb)[チュートリアル](federated_learning_for_text_generation.ipynb)を変更できます。

[FedAvg](http://proceedings.mlr.press/v54/mcmahan17a/mcmahan17a.pdf) アルゴリズムからのベースライン加重平均は、`tff.aggregators.MeanFactory` を使用して次のように表すことができます。

```
mean = tff.aggregators.MeanFactory()
iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
    ...,
    model_aggregator=mean)
```

このチュートリアルで説明されている加重平均を拡張するために使用できる手法は次のとおりです。

- ゼロイング
- クリッピング
- 差分プライバシー
- 圧縮
- セキュアアグリゲーション

拡張は構成を使用して行われます。この場合、`MeanFactory` は、集約の一部を委任する内部ファクトリをラップするか、それ自体が別の集約ファクトリによってラップされます。詳細については、[カスタムアグリゲータの実装](custom_aggregators.ipynb)チュートリアルを参照してください。

まず、これらの手法を個別に有効にして構成する方法を説明し、次にそれらを組み合わせる方法を示します。

## 手法

個々の手法を詳しく説明する前に、まず分位数マッチングアルゴリズムについて説明します。これは、以下の手法を構成するのに役立ちます。

### 分位数マッチング

以下の集約手法のいくつかでは、集約のいくつかの側面を制御するノルム境界を使用する必要があります。このような境界は定数として提供できますが、通常はトレーニングの過程で境界を調整することを推薦します。推奨される方法は、[Andrew et al. (2019)](https://arxiv.org/abs/1905.03871) の分位数マッチングアルゴリズムを使用することです。このアルゴリズムは当初は差分プライバシーとの互換性のために提案されましたが、より広く利用できます。特定の変位値での値を推定するには、`tff.aggregators.PrivateQuantileEstimationProcess` を使用できます。たとえば、分布の中央値に適応するには、次を使用できます。

In [ ]:
median_estimate = tff.aggregators.PrivateQuantileEstimationProcess.no_noise(
    initial_estimate=1.0, target_quantile=0.5, learning_rate=0.2)

分位数推定アルゴリズムを使用するさまざまな手法では、アルゴリズムパラメータのさまざまな値が必要になります。これについては、後で説明します。一般に、`learning_rate` パラメータを増やすと、正しい分位数への適応が速くなりますが、分散が大きくなります。`no_noise` クラスメソッドは、差分プライバシーのためのノイズを追加しない分位数マッチングプロセスを構築します。

### ゼロイング

ゼロイングとは、極端に大きな値をゼロに置き換えることです。 ここで、「極端に大きい」とは、事前定義されたしきい値よりも大きいこと、または前の計算ラウンドの値に比べて大きいことを意味します。ゼロイングにより、障害のあるクライアントでのデータ破損に対するシステムの堅牢性を高めることができます。

`ZEROING_CONSTANT` がゼロにされた L-無限大ノルムを持つ値の平均を計算するには、`tff.aggregators.MeanFactory` のゼロイングを実行する `tff.aggregators.zeroing_factory` でラップします。

```
zeroing_mean = tff.aggregators.zeroing_factory(
    zeroing_norm=MY_ZEROING_CONSTANT,
    inner_agg_factory=tff.aggregators.MeanFactory())
```

ここでは、`MeanFactory` を `zeroing_factory` でラップし、`zeroing_factory` の（事前集計）効果を、平均化による集計のために内部の `MeanFactory` に渡す前に、クライアントの値に適用します。

ただし、ほとんどのアプリケーションでは、分位数 Estimator を使用した適応ゼロイングを推薦します。そのためには、次のように分位数マッチングアルゴリズムを使用します。

In [ ]:
zeroing_norm = tff.aggregators.PrivateQuantileEstimationProcess.no_noise(
    initial_estimate=10.0,
    target_quantile=0.98,
    learning_rate=math.log(10),
    multiplier=2.0,
    increment=1.0)
zeroing_mean = tff.aggregators.zeroing_factory(
    zeroing_norm=zeroing_norm,
    inner_agg_factory=tff.aggregators.MeanFactory())

# Equivalent to:
# zeroing_mean = tff.learning.robust_aggregator(clipping=False)

パラメータは、プロセスがこれまでに見た最大値よりもいくらか大きい値に非常に迅速に（比較的大きい`learning_rate`）適応するように選択されています。分位数の推定値 `Q` の場合、ゼロイングに使用されるしきい値は `Q * multiplier + increment` になります。

### 境界 L2 ノルムへのクリッピング

クライアントの更新をクリッピングする（L2 ボールに投影する）と、外れ値に対する堅牢性を向上させることができます。`tff.aggregators.clipping_factory` は、上記の `tff.aggregators.zeroing_factory` とまったく同じように構成されており、`clipping_norm` 引数として定数または `tff.templates.EstimationProcess` のいずれかを取ることができます。次のように、適度に迅速に適度に高い基準に適応するクリッピングを使用することが推奨されています。

In [ ]:
clipping_norm = tff.aggregators.PrivateQuantileEstimationProcess.no_noise(
    initial_estimate=1.0,
    target_quantile=0.8,
    learning_rate=0.2)
clipping_mean = tff.aggregators.clipping_factory(
    clipping_norm=clipping_norm,
    inner_agg_factory=tff.aggregators.MeanFactory())

# Equivalent to:
# clipping_mean = tff.learning.robust_aggregator(zeroing=False)

私たちの経験では多くの問題では、学習率が適切に調整されている限り、`target_quantile` の値が正確であることはそれほど重要ではありません。ただし、非常に低く設定すると、クリッピングを使用しない場合と比較して、最高のパフォーマンスを得るためにサーバーの学習率を上げる必要がある場合があります。そのため、デフォルトでは 0.8 を推薦します。

### 差分プライバシー

TFF は、アダプティブクリッピングとガウスノイズを使用した差分プライベート集計もサポートします。差分プライベート平均化を実行するアグリゲータは、次のように構成できます。

In [ ]:
dp_mean = tff.aggregators.DifferentiallyPrivateFactory.gaussian_adaptive(
    noise_multiplier=0.1, clients_per_round=100)

# Equivalent to:
# dp_mean = tff.learning.dp_aggregator(
#   noise_multiplier=0.1, clients_per_round=100, zeroing=False)

`noise_multiplier` 引数の設定方法に関するガイダンスは、[TFF DP チュートリアル](https://www.tensorflow.org/federated/tutorials/federated_learning_with_differential_privacy)を参照してください。

### 非可逆圧縮

gzip などの可逆圧縮と比較すると、非可逆圧縮は一般にはるかに高い圧縮率をもたらし、後で可逆圧縮と組み合わせることができます。クライアントからサーバーへの通信に費やす時間が少ないため、トレーニングラウンドはより速く完了します。学習アルゴリズムの本質的にランダム化された性質により、あるしきい値までは、不可逆圧縮による不正確さが全体的なパフォーマンスに悪影響を与えることはありません。

デフォルトでは、テンソルサイズ圧縮 `threshold` と `quantization_bits` の 2 つの数値でパラメータ化された単純な均一量子化を使用することが推薦されています（[Suresh et al.](http://proceedings.mlr.press/v70/suresh17a/suresh17a.pdf) などを参照してください）。すべてのテンソル `t` について、`t` の要素数が `threshold` 以下の場合、圧縮されません。それを超える場合、`t` の要素は、`quantizaton_bits` ビットへのランダム化された丸めを使用して量子化されます。以下の演算を適用します。

`t = round((t - min(t)) / (max(t) - min(t)) * (2**quantizaton_bits - 1)),`

結果として、`[0, 2**quantizaton_bits-1]` の範囲の整数値になります。量子化された値は、送信のために整数型に直接パックされてから、逆変換が適用されます。

`quantizaton_bits` を 8 に設定し、`threshold` を 20000 に設定することを推薦します。


In [ ]:
compressed_mean = tff.aggregators.MeanFactory(
    tff.aggregators.EncodedSumFactory.quantize_above_threshold(
        quantization_bits=8, threshold=20000))

# Equivalent to:
# compressed_mean = tff.learning.compression_aggregator(zeroing=False, clipping=False)

#### 調整の提案

`quantization_bits` と `threshold` パラメータは両方とも調整できます。各トレーニングラウンドに参加するクライアントの数も圧縮の効果に影響を与える可能性があります。

**しきい値。**一般的なレイヤー型のバイアスなど、要素の数が少ない変数は、導入されたノイズに対してはるかに敏感であることが観察されているため、デフォルト値は 20000 に選択されています。さらに、実際には要素数が少ない変数を圧縮しても得られるメリットはほとんどありません。圧縮されていないサイズが比較的小さいためです。

一部のアプリケーションでは、しきい値の選択を変更することが理にかなっている場合があります。たとえば、分類モデルの出力層のバイアスは、ノイズに対してより敏感である可能性があります。語彙が 20004 の言語モデルをトレーニングしている場合は、` threshold`を 20004 に設定することを推薦します

**量子化ビット。**ほとんどのユーザーにとって `quantization_bits` のデフォルト値 8 は、問題ないはずです。8 がうまく機能していて、もう少しパフォーマンスを改善したい場合は、7 または 6 に下げてみてください。小さなグリッド検索を実行できる場合は、トレーニングが不安定になるか、最終的なモデルの品質が低下し始める値を特定し、その値から 2 つ上げることを推薦します。たとえば、`quantization_bits` を 5 に設定した場合に機能し、4 に設定するとモデルが劣化する場合は、デフォルトでは「安全側」になるように 6 に設定することを推薦します。

**ラウンドあたりのクライアント。**ラウンドあたりのクライアント数を大幅に増やすと、`quantization_bits` の値を小さくすることができます。これは、量子化によって生じるランダム化された不正確さが、より多くのクライアント更新を平均することで均等化される可能性があるためです。

### セキュアアグリゲーション

セキュアアグリゲーション（SecAgg）とは、サーバーが集計のみを復号化できるようにクライアントの更新が暗号化される暗号化プロトコルです。報告するクライアントの数が不十分な場合、サーバーは何も学習しません。また、サーバーは個々の更新を検査することはできません。これには、`tff.federated_secure_sum_bitwidth` 演算子を使用します。

モデルの更新は浮動小数点値ですが、SecAgg は整数で動作します。そのため、整数型に離散化する前に、大きな値をある境界内にクリップする必要があります。クリップの境界は、定数または適応的（推奨されるデフォルト）に決定することができます。その後、整数はセキュアに集計され、集計は浮動小数点ドメインにマップされます。

`MY_SECAGG_BOUND` をクリップの境界とし、SecAgg を使用して集計された加重値を使用して平均を計算するには、`SecureSumFactory` を `MeanFactory` に次のように渡します。

```
secure_mean = tff.aggregators.MeanFactory(
    tff.aggregators.SecureSumFactory(MY_SECAGG_BOUND))
```

境界を適応的に決定する場合は、次の通りです。

In [ ]:
secagg_bound = tff.aggregators.PrivateQuantileEstimationProcess.no_noise(
    initial_estimate=50.0,
    target_quantile=0.95,
    learning_rate=1.0,
    multiplier=2.0)
secure_mean = tff.aggregators.MeanFactory(
    tff.aggregators.SecureSumFactory(secagg_bound))

# Equivalent to:
# secure_mean = tff.learning.secure_aggregator(zeroing=Fasle, clipping=False)

#### 調整の提案

適応的なパラメータは、境界が厳密になるように選択されていますが（離散化の精度がそれほど失われることはありません）、クリップが発生することはめったにありません。

パラメータを調整する場合は、SecAgg プロトコルは平均で重み付けした後、重み付けされたモデルの更新を集計していることに注意してください。重みは通常、ローカルで処理されるデータポイントの数であるため、異なるタスク間では、右の境界はこの数値に依存する可能性があります。

適応的な `secagg_bound` を作成する場合、`increment` キーワード引数を使用することは推薦しません。これは、実際の推計が小さい場合、相対精度が大幅に低下する可能性があるためです。

上記のコードスニペットは、加重値のみを SecAgg に使用します。 重みの集計にも SecAgg を使用する必要がある場合は、一般的なトレーニングセットアップのように、境界を定数として設定することを推薦します。最大の重みが事前にわかっています。

```
secure_mean = tff.aggregators.MeanFactory(
    value_sum_factory=tff.aggregators.SecureSumFactory(secagg_bound),
    weight_sum_factory=tff.aggregators.SecureSumFactory(
        upper_bound_threshold=MAX_WEIGHT, lower_bound_threshold=0.0))
```

## 構成の手法

前述の平均を拡張するための個々の手法は、組み合わせることができます。

これらの手法をクライアントに適用するために推薦される順序は次の通りです。

1. ゼロイング
2. クリッピング
3. その他の手法

`tff.aggregators` モジュールのアグリゲータは、「内部アグリゲータ」（事前アグリゲータ効果が最後に発生し、事後アグリゲータ効果が最初に発生する）を「外部アグリゲータ」内にラップすることで構成されます。たとえば、ゼロイング、クリッピング、および圧縮を（この順序で）実行するには、次のように記述します。

```
# Compression is innermost because its pre-aggregation effects are last.
compressed_mean = tff.aggregators.MeanFactory(
    tff.aggregators.EncodedSumFactory.quantize_above_threshold(
        quantization_bits=8, threshold=20000))
# Compressed mean is inner aggregator to clipping...
clipped_compressed_mean = tff.aggregators.clipping_factory(
    clipping_norm=MY_CLIPPING_CONSTANT,
    inner_agg_factory=compressed_mean)
# ...which is inner aggregator to zeroing, since zeroing happens first.
final_aggregator = tff.aggregators.zeroing_factory(
    zeroing_norm=MY_ZEROING_CONSTANT,
    inner_agg_factory=clipped_compressed_mean)
```

この構造は、アルゴリズムを学習するための[デフォルトのアグリゲータ](https://github.com/tensorflow/federated/blob/11e4f632b38745c9b38cc39fa1fe67771c206e77/tensorflow_federated/python/learning/model_update_aggregator.py)と一致することに注意してください。

他の構成も可能です。複数の異なるアプリケーションで機能するデフォルト構成が確認されたら、このドキュメントを拡張します。新しいアイデアの実装については、[カスタムアグリゲータの実装](custom_aggregators.ipynb)チュートリアルを参照してください。